# NLP Analysis of Scientific Abstracts
---
####  US National Library of Medicine National Institutes of Health 
#### The National Center for Biotechnology Information 
#### PubMed

In [1]:
# Import custom module for calling abstracts from a key word or phrase
from lib.get_abstracts import get_abstracts

In [2]:
# Import standard analysis tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Import tools for NLP 
import scipy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim

In [4]:
# Get abstracts and create csv of abstracts
# API key obtained from NCBI
#  get_abstracts('gene therapy', 1000, 'ragorham1@gmail.com', 
#                '3ffbbb6bd110815d69e4aa14b7c26d72ab09')

In [12]:
# Process abstracts 
df = pd.read_csv('./data/gene therapy.csv', header=-1, 
                 names=['pmid', 'term', 'abstract'])
df = df.dropna()

abstracts = list(df.abstract)

# Make tf-idf matrix
vectorizer = TfidfVectorizer(stop_words='english')
abstracts_tfidf = vectorizer.fit_transform(abstracts)

In [26]:
import guidedlda
from scipy import sparse
xx = sparse.coo_matrix(abstracts)
# X = guidedlda.datasets.load_data(xx)
# vocab = guidedlda.datasets.load_vocab(abstracts)
# word2id = dict((v, idx) for idx, v in enumerate(vocab))

xx.shape

(1, 98)

In [13]:
#  Make the wors list
terms = vectorizer.get_feature_names()

# Number of topics
ntopics=10

# Linking words to topics
def word_topic(tfidf,solution, wordlist):
    
    # Loading scores for each word on each topic/component.
    words_by_topic=tfidf.T * solution

    # Linking the loadings to the words in an easy-to-read way.
    components=pd.DataFrame(words_by_topic,index=wordlist)
    
    return components
# Extracts the top N words and their loadings for each topic.
def top_words(components, n_top_words):
    n_topics = range(components.shape[1])
    index= np.repeat(n_topics, n_top_words, axis=0)
    topwords=pd.Series(index=index)
    for column in range(components.shape[1]):
        # Sort the column so that highest loadings are at the top.
        sortedwords=components.iloc[:,column].sort_values(ascending=False)
        # Choose the N highest loadings.
        chosen=sortedwords[:n_top_words]
        # Combine loading and index into a string.
        chosenlist=chosen.index +" "+round(chosen,2).map(str) 
        topwords.loc[column]=chosenlist
    return(topwords)

# Number of words to look at for each topic.
n_top_words = 10

In [14]:
svd= TruncatedSVD(ntopics)
lsa = make_pipeline(svd, Normalizer(copy=False))
abstracts_lsa = lsa.fit_transform(abstracts_tfidf)

components_lsa = word_topic(abstracts_tfidf, abstracts_lsa, terms)

topwords=pd.DataFrame()
topwords['LSA']=top_words(components_lsa, n_top_words)    

In [8]:
lda = LDA(n_topics=ntopics, 
          doc_topic_prior=None, # Prior = 1/n_documents
          topic_word_prior=1/ntopics,
          learning_decay=0.7, # Convergence rate.
          learning_offset=10.0, # Causes earlier iterations to have less influence on the learning
          max_iter=10, # when to stop even if the model is not converging (to prevent running forever)
          evaluate_every=-1, # Do not evaluate perplexity, as it slows training time.
          mean_change_tol=0.001, # Stop updating the document topic distribution in the E-step when mean change is < tol
          max_doc_update_iter=100, # When to stop updating the document topic distribution in the E-step even if tol is not reached
          n_jobs=-1, # Use all available CPUs to speed up processing time.
          verbose=0, # amount of output to give while iterating
          random_state=0
         )

abstracts_lda = lda.fit_transform(abstracts_tfidf) 

components_lda = word_topic(abstracts_tfidf, abstracts_lda, terms)

topwords['LDA']=top_words(components_lda, n_top_words)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [19]:
abstracts_tfidf.data

array([0.15816771, 0.15816771, 0.15816771, ..., 0.1973093 , 0.1973093 ,
       0.1973093 ])

In [9]:
nmf = NMF(alpha=0.0, 
          init='nndsvdar', # how starting value are calculated
          l1_ratio=0.0, # Sets whether regularization is L2 (0), L1 (1), or a combination (values between 0 and 1)
          max_iter=200, # when to stop even if the model is not converging (to prevent running forever)
          n_components=ntopics, 
          random_state=0, 
          solver='cd', # Use Coordinate Descent to solve
          tol=0.0001, # model will stop if tfidf-WH <= tol
          verbose=0 # amount of output to give while iterating
         )
abstracts_nmf = nmf.fit_transform(abstracts_tfidf) 

components_nmf = word_topic(abstracts_tfidf, abstracts_nmf, terms)

topwords['NNMF']=top_words(components_nmf, n_top_words)

In [10]:
# for topic in range(ntopics):
#     print('Topic {}:'.format(topic))
#     print(topwords.loc[topic])

In [11]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, abstracts_tfidf, vectorizer, R=10, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
5      20.693446        1       1  124.364807   -3.041184
8      19.060554        1       2   31.463371   68.117889
3      13.012010        1       3  -56.127460  146.906677
2      11.522483        1       4  145.245880   98.682457
1       9.065029        1       5  -15.510025 -128.050049
0       8.942207        1       6   96.903442 -103.188919
9       8.417129        1       7 -104.470146  -54.967331
6       4.873226        1       8  -83.596214   46.759289
7       2.224770        1       9    9.307807  -24.400661
4       2.189147        1      10   56.286880  171.765549, topic_info=     Category      Freq              Term     Total  loglift  logprob
term                                                                 
1326  Default  0.000000               gba  0.000000  10.0000  10.0000
1936  Default  0.000000          melanoma  0.000000   9.0000   9.0000
433   Default  0.000000           bladder  0.000000   8.0000   8.0000
2316  Default  0.000000                pd  0.000000   7.0000   7.0000
322   Default  0.000000             areas  0.000000   6.0000   6.0000
3051  Default  0.000000           subtype  0.000000   5.0000   5.0000
3007  Default  0.000000            steap1  0.000000   4.0000   4.0000
1115  Default  0.000000   epidemiological  0.000000   3.0000   3.0000
3337  Default  0.000000          variants  0.000000   2.0000   2.0000
571   Default  0.000000              chd1  0.000000   1.0000   1.0000
3362   Topic1  0.127314             viral  0.292654   0.7430  -6.9890
3343   Topic1  0.139669            vector  0.328786   0.7192  -6.8964
3346   Topic1  0.102121              vein  0.250509   0.6780  -7.2095
787    Topic1  0.104173            crispr  0.255712   0.6774  -7.1896
880    Topic1  0.130187          delivery  0.323237   0.6659  -6.9667
1917   Topic1  0.096284               mdx  0.239266   0.6651  -7.2684
3344   Topic1  0.135156           vectors  0.336225   0.6640  -6.9292
2466   Topic1  0.086910           prevent  0.222854   0.6337  -7.3708
1392   Topic1  0.092784            gucy2c  0.246734   0.5973  -7.3054
2542   Topic1  0.087378          protocol  0.234762   0.5870  -7.3654
1331   Topic1  0.152113              gene  0.418270   0.5639  -6.8111
3167   Topic1  0.118422           therapy  0.361834   0.4584  -7.0614
540    Topic1  0.102567             cells  0.357002   0.3281  -7.2052
524    Topic2  0.096274           catenin  0.246045   0.7192  -7.1863
3039   Topic2  0.103490               sub  0.264923   0.7176  -7.1140
550    Topic2  0.091960          cervical  0.240418   0.6965  -7.2321
2649   Topic2  0.079085             redox  0.214911   0.6578  -7.3830
1317   Topic2  0.087972               gan  0.239084   0.6578  -7.2765
446    Topic2  0.082073                bm  0.227164   0.6395  -7.3459
2607   Topic2  0.084240              rats  0.233867   0.6365  -7.3198
...       ...       ...               ...       ...      ...      ...
1245   Topic8  0.017084          fibrosis  0.189997   0.6126  -7.5515
765    Topic8  0.017966       correlation  0.203970   0.5919  -7.5012
2531   Topic8  0.016728          prostate  0.191832   0.5819  -7.5726
2147   Topic8  0.016785      neuroscience  0.192505   0.5818  -7.5691
1326   Topic9  0.010663               gba  0.188371   0.9339  -7.2388
2316   Topic9  0.009071                pd  0.202419   0.7003  -7.4004
2011   Topic9  0.006800          modeling  0.185613   0.4988  -7.6885
1855   Topic9  0.005999         lysosomal  0.183521   0.3848  -7.8139
194    Topic9  0.005848        advantages  0.181547   0.3701  -7.8394
1215   Topic9  0.005451       facilitated  0.169543   0.3683  -7.9096
1703   Topic9  0.006243          involved  0.196128   0.3582  -7.7741
375    Topic9  0.005669        autonomous  0.181679   0.3383  -7.8705
883    Topic9  0.005716          dementia  0.185192   0.3273  -7.8623
206    Topic9  0.005374            ageing